<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/TOLD_BR_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing needed libraries
!pip install -U scikit-learn
!pip install -U spacy
!python -m spacy download pt


!pip install wandb
!pip install --upgrade simpletransformers
!pip install --upgrade transformers

from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter
import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.model_selection import train_test_split
from sklearn.metrics._plot import confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import numpy as np
import sklearn
import pickle
import spacy
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import pipeline
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

BERT IMBAU

In [10]:
told_br = pd.read_csv("/content/drive/MyDrive/PIC /Dataset/told_br_binary.csv")

Tranformando a base em binária

In [11]:
told_br["toxic"] = 0
told_br["toxic_1"] = 0
told_br["toxic_2"] = 0
told_br["toxic_3"] = 0
told_br["homophobia"] = 0
told_br["obscene"] = 0
told_br["insult"] = 0
told_br["racism"] = 0
told_br["misogyny"] = 0
told_br["xenophobia"] = 0
for i, row in told_br.iterrows():
    count = 0
    if 1 in list(row[["homophobia_1", "obscene_1", "insult_1", "racism_1", "misogyny_1", "xenophobia_1"]]):
        told_br.loc[i, "toxic_1"] = 1
        count += 1
    if 1 in list(row[["homophobia_2", "obscene_2", "insult_2", "racism_2", "misogyny_2", "xenophobia_2"]]):
        told_br.loc[i, "toxic_2"] = 1
        count += 1
    if 1 in list(row[["homophobia_3", "obscene_3", "insult_3", "racism_3", "misogyny_3", "xenophobia_3"]]):
        told_br.loc[i, "toxic_3"] = 1
        count += 1

    told_br.loc[i, "homophobia"] = np.sum(list(row[["homophobia_1", "homophobia_2", "homophobia_3"]]))

    told_br.loc[i, "obscene"] = np.sum(list(row[["obscene_1", "obscene_2", "obscene_3"]]))

    told_br.loc[i, "insult"] = np.sum(list(row[["insult_1", "insult_2", "insult_3"]]))

    told_br.loc[i, "racism"] = np.sum(list(row[["racism_1", "racism_2", "racism_3"]]))

    told_br.loc[i, "misogyny"] = np.sum(list(row[["misogyny_1", "misogyny_2", "misogyny_3"]]))

    told_br.loc[i, "xenophobia"] = np.sum(list(row[["xenophobia_1", "xenophobia_2", "xenophobia_3"]]))

    if count >= 1:
        told_br.loc[i, "toxic"] = 1

In [12]:
data_told = told_br[["text", "toxic", "homophobia", "obscene", "insult", "racism", "misogyny", "xenophobia"]]

Treinamento e preparação dos dados

In [13]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
data_told = data_told.iloc[:,[0, 1]]
train,eva = train_test_split(data_told,test_size = 0.2)


train_df = pd.DataFrame({
    'text': train['text'],
    'label': train['toxic']
})

eval_df = pd.DataFrame({
    'text': eva['text'],
    'label': eva['toxic']
})

#Optional model configuration
model_args = ClassificationArgs(
  num_train_epochs=1,
  overwrite_output_dir=True
                                )
# #Create a ClassificationModel
# model = ClassificationModel(
#     "bert", "neuralmind/bert-base-portuguese-cased", args=model_args,
#     use_cuda=False
# )

# model.train_model(train_df)

# result, model_outputs, wrong_predictions = model.eval_model(eval_df)
# lst = []
# for arr in model_outputs:
#     lst.append(np.argmax(arr))
# true = eval_df['label'].tolist()
# predicted = lst
# print(classification_report(true, predicted))

#Save model
# with open('TOLD_BR-BERT.pkl ', 'wb') as file:
#     pickle.dump(model, file)


Código para carregar modelo

In [14]:
with open('/content/drive/MyDrive/PIC /Models/TOLD_BR-BERT.pkl_', 'rb') as f:
    model = pickle.load(f)

validação do modelo

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eval_df['label'].tolist()
predicted = lst

In [31]:
print(classification_report(true, predicted))

              precision    recall  f1-score   support

           0       0.87      0.79      0.83      2334
           1       0.77      0.85      0.81      1866

    accuracy                           0.82      4200
   macro avg       0.82      0.82      0.82      4200
weighted avg       0.82      0.82      0.82      4200



In [34]:
confusion_matrix(true, predicted)

array([[1848,  486],
       [ 275, 1591]])

Função para pré-processamento

In [ ]:
npl = spacy.load("pt_core_news_sm")
def preprocess(text):
  comentarios = text.lower()
  only_latters =re.sub("[^a-zA-Z]", " ", comentarios) # somente letras
  doc = npl(comentarios)
  filtered_result = []
  for token in doc:
    if token.is_stop or token.is_punct :
      continue
    filtered_result.append(token.lemma_)
  return " ".join(filtered_result)

told_br['preprocessed_comments'] =  told_br['text'].apply(preprocess)